# Project Report #
## About Dataset ##
I decided to choose the [Titanic data](https://www.udacity.com/api/nodes/5420148578/supplemental_media/titanic-datacsv/download) just out of curiosity.
Data contains the demographics and passenger information from 891 of the 2224 passengers and crew on board the Titanic. Description of the data can be found [here](https://www.kaggle.com/c/titanic/data)

## Questions  ##
* How the survival of the titanic boat passenger is related to his sex, age ?
* Do other factors like **sibsp**(Number of Siblings/Spouses Aboard), **parch**(Number of Parents/Children Aboard), **pcalss**(Passenger Class) affect the survival rate ?



## Data Exploration ##
## Conclusions ##


## Data Wrangling  ##


In [17]:
# import data from csv
import csv
data_file = 'titanic_data.csv'


def read_data(filename):
    with open(filename,'rb') as f:
        return csv.DictReader(f);
    
titanic_data_values = read_data(data_file)
print 'hello'
header = next(titanic_data_values)
print header

hello


ValueError: I/O operation on closed file